In [1]:
%load_ext twisted_ipython
%autoawait twisted

In [2]:
from pyxsession.xdg.autostart import AutostartConfiguration
from pyxsession.config import load_config

In [3]:
autostart = AutostartConfiguration(load_config())

In [4]:
autostart.autostart_entries

{'gnome-shell-overrides-migration.desktop': AutostartEntry(
     fullpath='/etc/xdg/autostart/gnome-shell-overrides-migration.desktop',
     filename='gnome-shell-overrides-migration.desktop',
     overrides=[],
     entry=<xdg.DesktopEntry.DesktopEntry at 0x7f5f74714160>,
     parsed=True,
     validated=True,
     exec_parsed=True,
     exec=<pyxsession.xdg.exec_key.ExecKey at 0x7f5f65cca320>,
     is_application=True,
     is_hidden=False,
     only_show_in=[],
     not_show_in=[],
     dbus_activatable=False,
     runtime_path=None),
 'emoji-keyboard.desktop': AutostartEntry(
     fullpath='/etc/xdg/autostart/emoji-keyboard.desktop',
     filename='emoji-keyboard.desktop',
     overrides=[],
     entry=<xdg.DesktopEntry.DesktopEntry at 0x7f5f65cd1d68>,
     parsed=True,
     validated=True,
     exec_parsed=True,
     exec=<pyxsession.xdg.exec_key.ExecKey at 0x7f5f65cd1fd0>,
     is_application=True,
     is_hidden=False,
     only_show_in=[],
     not_show_in=[],
     dbus_activat

In [5]:
from pyxsession.xdg.exec_key import ExecKey

In [6]:
key = ExecKey('nautilus %f')

In [7]:
key.build_argv(dict(f='./my-directory'))

['nautilus', './my-directory']